In [1]:
import tensorflow as tf
import os
import numpy as np

In [2]:
pose_train_data_dir = r"E:\MasterDaten\Datasets\FHAD"


In [3]:
list_ds = tf.data.Dataset.list_files(os.path.join(pose_train_data_dir,'*/skeleton.txt'))
for f in list_ds.take(5):
    print(f.numpy())
    

b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_5\\toast_wine\\3\\skeleton.txt'
b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_3\\take_letter_from_enveloppe\\1\\skeleton.txt'
b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_6\\tear_paper\\1\\skeleton.txt'
b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_2\\scratch_sponge\\1\\skeleton.txt'
b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_3\\write\\3\\skeleton.txt'


In [30]:
@tf.function
def parse_skeleton(skeleton_data_line, file_path):
    data_elems = tf.strings.split(skeleton_data_line, " ")
    return file_path, data_elems[0], tf.strings.to_number(data_elems[1:], out_type=tf.dtypes.float32)

In [31]:
# test:
file_path = "E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/charge_cell_phone/1/skeleton.txt"
simple_skeleton_dataset = tf.data.TextLineDataset(file_path).map(lambda x: parse_skeleton(x, file_path))
for skeleton in simple_skeleton_dataset.take(1):
    print(skeleton)
    

(<tf.Tensor: id=717, shape=(), dtype=string, numpy=b'E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/charge_cell_phone/1/skeleton.txt'>, <tf.Tensor: id=718, shape=(), dtype=string, numpy=b'0000'>, <tf.Tensor: id=719, shape=(63,), dtype=float32, numpy=
array([ 37.4739,  76.0712, 348.356 ,  32.5099,  64.3266, 339.0773,
        35.5006,  16.1834, 401.3612,  40.1808,  34.2762, 412.9691,
        43.7768,  52.4597, 417.1439,  47.0601,  67.0314, 423.2051,
         7.4596,  51.6898, 389.6903,  -9.1929,  43.2894, 423.3356,
       -16.5373,  33.5237, 448.5287,   8.367 ,  27.0099, 436.9117,
       -11.5193,  34.9446, 462.9667, -28.2976,  45.3796, 466.0647,
        25.9924,  31.0274, 459.7573,  12.739 ,  31.2294, 486.6702,
        -4.6451,  35.2024, 502.7328,  37.3626,  38.9797, 458.534 ,
        13.1163,  45.1006, 471.1297,  -7.6845,  49.4089, 484.3117,
        34.9216,  66.9734, 452.8134,  17.1006,  72.8127, 455.8657,
        -3.5018,  79.5218, 459.6773], dtype=float32)>)


In [6]:
@tf.function
def read_skeleton_file(file_path):
    skeleton_vals = tf.data.TextLineDataset(file_path).map(lambda x: parse_skeleton(x, file_path))
    return skeleton_vals

In [11]:
skeleton_data = list_ds.flat_map(read_skeleton_file)
for skeleton in skeleton_data.take(1):
    print(skeleton)

(<tf.Tensor: id=289, shape=(), dtype=string, numpy=b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_1\\close_liquid_soap\\4\\skeleton.txt'>, <tf.Tensor: id=290, shape=(), dtype=string, numpy=b'0000'>, <tf.Tensor: id=291, shape=(63,), dtype=string, numpy=
array([b'77.481400', b'-5.575300', b'331.042400', b'75.168800',
       b'-10.791900', b'316.341700', b'67.812400', b'-83.710500',
       b'345.233900', b'69.029900', b'-74.157300', b'365.014100',
       b'70.715500', b'-60.745100', b'378.365900', b'71.898900',
       b'-51.378600', b'391.431900', b'46.846500', b'-57.218700',
       b'324.551400', b'38.594000', b'-89.737100', b'319.033000',
       b'37.220200', b'-116.002400', b'309.428500', b'46.438100',
       b'-114.970000', b'367.804500', b'31.388700', b'-136.979500',
       b'383.696300', b'16.581200', b'-147.051600', b'392.600800',
       b'45.207200', b'-108.452200', b'393.558100', b'29.891800',
       b'-130.500200', b'411.908800', b'12.142400', b'-141.388200'

In [23]:
@tf.function
def get_corresponding_images(skeleton_path, frame_index, skeleton):
    video_path = tf.strings.regex_replace(skeleton_path, "Hand_pose_annotation_v1", "Video_files")
    path_length = tf.strings.length(video_path)
    video_path = tf.strings.substr(video_path, 0, path_length - len("skeleton.txt"))
    path_add = tf.constant("depth/depth_")
    file_ending = tf.constant(".png")
    img_path = tf.strings.join([video_path, path_add, frame_index, file_ending])
    return skeleton_path, tf.io.decode_png(tf.io.read_file(img_path), dtype=tf.dtypes.uint16), skeleton


In [24]:
full_ds = skeleton_data.map(get_corresponding_images)
for path, image_raw, skeleton in full_ds.take(1):
    print(path.numpy())
    print()
    print(repr(image_raw.numpy()[:100]))
    print()
    print(skeleton.numpy())

b'E:\\MasterDaten\\Datasets\\FHAD\\Hand_pose_annotation_v1\\Subject_4\\read_letter\\2\\skeleton.txt'

array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=uint16)

[b'9.538700' b'23.072500' b'270.988000' b'-0.768100' b'21.468100'
 b'262.173500' b'-17.322400' b'-39.127200' b'292.217400' b'-2.153600'
 b'-32.258900' b'305.013400' b'10.991300' b'-24.877300' b'314.848100'
 b'23.990900' b'-13.010100' b'320.568000' b'-36.378900' b'2.157900'
 b'300.991300' b'-60.320500' b'-10.824